In [1]:
from langgraph.graph import StateGraph,START,END
from langchain_groq import ChatGroq
from typing import TypedDict,Literal,Annotated
from dotenv import load_dotenv
from pydantic import BaseModel,Field

In [2]:
load_dotenv()

True

In [3]:
model=ChatGroq(model="llama-3.1-8b-instant")

In [4]:
class StateSchema(TypedDict):
    topic:str
    notes:str
    questions:str
    status:Literal["approved","rejected"]
    score:float
    feedback:str
    iterator:int

In [5]:
class NotesSchema(BaseModel):
    notes:Annotated[str,Field(description="200 words summary of the given topic")]
    questions:Annotated[str,Field(description="3 MCQ questions that test the most important concepts for the given topic")]

class EvaluationSchema(BaseModel):
    status:Literal["approved","rejected"]
    feedback:Annotated[str,Field(description="Provide the feeback about the notes")]
    score:Annotated[float,Field(description="Rate the score out of 5")]

In [6]:
structured_model1=model.with_structured_output(NotesSchema)
structured_model2=model.with_structured_output(EvaluationSchema)

In [7]:
def generate_notes(state:StateSchema)->StateSchema:

    prompt=f"You are my example helper. I will give you a topic and you have to generate notes for the things that I must remember so that I get good grades in my exame. It should not be longer than 200 words and avoid mathematical derivation, just summary of the topic that I give you.Also give me 3 MCQ questions that test the most important concepts for the given topic:{state['topic']}"

    result=structured_model1.invoke(prompt)

    return {
        "notes": result.notes,
        "questions":result.questions,
        "iterator":0
    }
    

In [8]:
def evaluate_notes(state:StateSchema):

    prompt=f"This is a very tricky situtation. Say I have not studied for my exams and this is the only shot I have got. I'll provide you with a summary of the notes that I have. Can you accept or reject the notes based on if it provides an detailed overview of the given topic which will help me pass an exam. Also provide me a score on the notes and feedback on it: {state['notes']}"

    result=structured_model2.invoke(prompt)

    return {
        "status": result.status,
        "feedback":result.feedback,
        "score":result.score
    }

def optimize_notes(state:StateSchema):
    prompt=f"Can you optimize the given notes:{state['notes']} based on the given feedback:{state['feedback']}. You can summarize it within 500 words and also provide 3 MCQ questions that best reflect the given topic"
    result=structured_model1.invoke(prompt)
    iteration=state["iterator"]+1

    return {
        "notes": result.notes,
        "questions":result.questions,
        "iterator":iteration
    }

def conditional_edge(state:StateSchema):
    if state["status"]=="approved" or state["iterator"]==5:
        return "approved"
    else:
        return "rejected"



In [9]:
graph=StateGraph(StateSchema)

In [10]:
graph.add_node('generate',generate_notes)
graph.add_node('evaluate',evaluate_notes)
graph.add_node('optimize',optimize_notes)

graph.add_edge(START,'generate')
graph.add_edge('generate','evaluate')

graph.add_conditional_edges('evaluate',conditional_edge,{'approved':END,'rejected':'optimize'})
graph.add_edge('optimize',END)


In [11]:
workflow=graph.compile()
workflow

ValueError: Failed to reach https://mermaid.ink API while trying to render your graph. Status code: 400.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [12]:
initial_state={
    'topic':'Convolution Neural Network'
}
workflow.invoke(initial_state)

{'topic': 'Convolution Neural Network',
 'notes': 'A Convolutional Neural Network (CNN) is a type of deep learning model that uses convolutional and pooling layers to analyze data with grid-like topology. It is commonly used in image and video processing tasks. The key concept in CNN is the convolutional layer, which applies a filter to a small region of the input data, producing a feature map. The pooling layer reduces the spatial dimensions of the feature map, helping to reduce the number of parameters and prevent overfitting. CNNs are particularly useful for tasks like image classification, object detection, and image segmentation. The key components of a CNN include convolutional layers, pooling layers, fully connected layers, and activation functions like ReLU and Sigmoid.',
 'questions': '1. What is the primary function of a convolutional layer in a CNN? A) To apply a filter to a small region of the input data B) To reduce the spatial dimensions of the feature map C) To classify 